# Fase 2: Limpieza de Datos

In [723]:
import pandas as pd
import numpy as np
import json

import sys
import os

# Agregar el directorio 'src' al path
sys.path.append(os.path.abspath('../src'))

# Importar la función suma del módulo auxiliares
import soporte_limpieza as sl

In [724]:
df = pd.read_csv("../datos/tablas_concatenadas.csv", parse_dates=["fecha recaudacion"])
df.head(2)

,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
0,0,63000.0,NaN,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",0.0,0.0,1297.13,0.0,2013-12-31,2013.0
1,1,63000.0,Advocacia-Geral da União,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",0.0,0.0,26666621.42,0.0,2013-12-31,2013.0


In [725]:
df.isnull().sum()

Unnamed: 0                       0
id organizacion superior     30359
organizacion superior       359187
id organizacion              25117
organizacion                 34886
id unidad gestora            33581
unidad gestora               19481
categoria economica          18978
origen ingreso               38417
tipo ingreso                 31927
valor previsto               51315
valor registrado             26419
valor recaudado              39442
porcentaje recaudado         24134
fecha recaudacion            23747
anio recaudacion                 0
dtype: int64

### 1- Gestión de los nulos de id organizacion superior y organizacion superior.

Vamos a ver si podemos crear un diccionario para los id y los nombres de la organización superior.

In [726]:
len(df["id organizacion superior"].unique())

26

In [727]:
len(df["organizacion superior"].unique())

26

A priori parece que hay el mismo numero de ids que de nombres, buena señal.

In [728]:
lista_id_os = list(df["id organizacion superior"].unique())
lista_id_os

[np.float64(63000.0),
 np.float64(37000.0),
 np.float64(nan),
 np.float64(22000.0),
 np.float64(55000.0),
 np.float64(24000.0),
 np.float64(52000.0),
 np.float64(25000.0),
 np.float64(26000.0),
 np.float64(39000.0),
 np.float64(30000.0),
 np.float64(81000.0),
 np.float64(58000.0),
 np.float64(33000.0),
 np.float64(36000.0),
 np.float64(41000.0),
 np.float64(57000.0),
 np.float64(35000.0),
 np.float64(32000.0),
 np.float64(49000.0),
 np.float64(53000.0),
 np.float64(51000.0),
 np.float64(44000.0),
 np.float64(38000.0),
 np.float64(54000.0),
 np.float64(20000.0)]

In [729]:
for id in lista_id_os:
    print(f"\n{id}")
    filtro1 = df["id organizacion superior"] == float(id)
    filtro2 = df["organizacion superior"].notnull()
    print(len(df[filtro1 & filtro2]["organizacion superior"].unique()))


63000.0
1

37000.0
1

nan
0

22000.0
1

55000.0
1

24000.0
1

52000.0
1

25000.0
1

26000.0
1

39000.0
1

30000.0
1

81000.0
1

58000.0
1

33000.0
1

36000.0
1

41000.0
1

57000.0
1

35000.0
1

32000.0
1

49000.0
1

53000.0
1

51000.0
1

44000.0
1

38000.0
1

54000.0
1

20000.0
1


Como vemos que para cada id hay un unico valor creamos un diccionario:

In [730]:
dic_id_os = {}
for id in lista_id_os:
    try:
        filtro1 = df["id organizacion superior"] == float(id)
        filtro2 = df["organizacion superior"].notnull()
        dic_id_os[float(id)]=df[filtro1 & filtro2]["organizacion superior"].unique()[0]
    except:
        pass

In [731]:
dic_id_os

{63000.0: 'Advocacia-Geral da União',
 37000.0: 'Controladoria-Geral da União',
 22000.0: 'Ministério da Agricultura, Pecuária e Abastec',
 55000.0: 'Ministério da Cidadania',
 24000.0: 'Ministério da Ciência, Tecnologia, Inovações ',
 52000.0: 'Ministério da Defesa',
 25000.0: 'Ministério da Economia',
 26000.0: 'Ministério da Educação',
 39000.0: 'Ministério da Infraestrutura',
 30000.0: 'Ministério da Justiça e Segurança Pública',
 81000.0: 'Ministério da Mulher, Família e Direitos Huma',
 58000.0: 'Ministério da Pesca e Aquicultura',
 33000.0: 'Ministério da Previdência Social',
 36000.0: 'Ministério da Saúde',
 41000.0: 'Ministério das Comunicações',
 57000.0: 'Ministério das Mulheres, Igualdade Racial, da',
 35000.0: 'Ministério das Relações Exteriores',
 32000.0: 'Ministério de Minas e Energia',
 49000.0: 'Ministério do Desenvolvimento Agrário',
 53000.0: 'Ministério do Desenvolvimento Regional',
 51000.0: 'Ministério do Esporte',
 44000.0: 'Ministério do Meio Ambiente',
 38000.

Ya con nuestro diccionario vamos a rellenar los nombres e id de las organizaciones superiores.

Hemos comprobado que todos los campos de organización superior concuerdan con su id a excepción de cuando son NaN, corregimos los NaN. además quedarán sin corregir aquellas filas donde id y organización superioir son ambos NaN.

In [732]:
# filas que van a quedar sin corregir.
filtro1 = df["id organizacion superior"].isnull()
filtro2 = df["organizacion superior"].isnull()
df[filtro1 & filtro2].shape

(10545, 16)

In [733]:
df["organizacion superior"].fillna(df["id organizacion superior"].map(dic_id_os), inplace=True)

C:\Users\Elena\AppData\Local\Temp\ipykernel_13844\3684483159.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["organizacion superior"].fillna(df["id organizacion superior"].map(dic_id_os), inplace=True)


In [734]:
os_null = df[df["organizacion superior"].isnull()].shape[0]
os_sin_corregir = os_null/df.shape[0]*100
print(f"Han quedado a null {os_null}, es decir, un {np.round(os_sin_corregir,2)}% del total de las organizaciones superiores.")

Han quedado a null 10545, es decir, un 1.03% del total de las organizaciones superiores.


In [735]:
df[df["id organizacion superior"].isnull()].shape[0]

30359

Como podemos ver ids hay 30359 a null los cuales podemos corregir al rededor de unos 20000 a partir del nombre de la organizacion superior.

In [736]:
#Invertimos el diccionario
dic_organizaciones_id = {valor: clave for clave, valor in dic_id_os.items()}
dic_organizaciones_id


{'Advocacia-Geral da União': 63000.0,
 'Controladoria-Geral da União': 37000.0,
 'Ministério da Agricultura, Pecuária e Abastec': 22000.0,
 'Ministério da Cidadania': 55000.0,
 'Ministério da Ciência, Tecnologia, Inovações ': 24000.0,
 'Ministério da Defesa': 52000.0,
 'Ministério da Economia': 25000.0,
 'Ministério da Educação': 26000.0,
 'Ministério da Infraestrutura': 39000.0,
 'Ministério da Justiça e Segurança Pública': 30000.0,
 'Ministério da Mulher, Família e Direitos Huma': 81000.0,
 'Ministério da Pesca e Aquicultura': 58000.0,
 'Ministério da Previdência Social': 33000.0,
 'Ministério da Saúde': 36000.0,
 'Ministério das Comunicações': 41000.0,
 'Ministério das Mulheres, Igualdade Racial, da': 57000.0,
 'Ministério das Relações Exteriores': 35000.0,
 'Ministério de Minas e Energia': 32000.0,
 'Ministério do Desenvolvimento Agrário': 49000.0,
 'Ministério do Desenvolvimento Regional': 53000.0,
 'Ministério do Esporte': 51000.0,
 'Ministério do Meio Ambiente': 44000.0,
 'Minis

In [737]:
df["id organizacion superior"].fillna(df["organizacion superior"].map(dic_organizaciones_id), inplace=True)

C:\Users\Elena\AppData\Local\Temp\ipykernel_13844\1809695679.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["id organizacion superior"].fillna(df["organizacion superior"].map(dic_organizaciones_id), inplace=True)


In [738]:
df[df["id organizacion superior"].isnull()].shape[0]

10545

Comprobamos cuantos null hay ahora y efectivamente se han reducido hasta 10545, los mismo que para los nombres de las organizaciones superiores ya que son todas las filas que tienen ambos campos a null. Aunque al representar un 1% y al no haber forma de arreglarlo puede ser despreciable.

In [739]:
df.isnull().sum()

Unnamed: 0                      0
id organizacion superior    10545
organizacion superior       10545
id organizacion             25117
organizacion                34886
id unidad gestora           33581
unidad gestora              19481
categoria economica         18978
origen ingreso              38417
tipo ingreso                31927
valor previsto              51315
valor registrado            26419
valor recaudado             39442
porcentaje recaudado        24134
fecha recaudacion           23747
anio recaudacion                0
dtype: int64

___

In [740]:
# pd.set_option("display.max_rows", None)
resultado = df.groupby(['id organizacion'])['id organizacion superior'].nunique().reset_index()
resultado[resultado["id organizacion superior"]!=1]

,id organizacion,id organizacion superior


Por lo tanto para cada par de id orgnización solo va a estar asociado a un único id organizacion superior. Creamos un diccionario con clave el id de organización y como valor tuplas de los valores de id organizacion superior y organizacion superior.

In [741]:
# Eliminamos aquellas filas donde haya nulos
df_validos = df.dropna(subset=['id organizacion superior', 'organizacion superior', 'id organizacion'])

diccionario_orga_a_sup = {}
for index, row in df_validos.iterrows():
    diccionario_orga_a_sup[row['id organizacion']] = (row['id organizacion superior'], row['organizacion superior'])

len(diccionario_orga_a_sup)

291

In [742]:
with open('../datos/diccionario_orga_a_sup.json', 'w') as file:
    json.dump(diccionario_orga_a_sup, file)

He intentado meter esta funcion en el archivo .py pero llevaba más de dos minutos ejecutandose y no terminaba, en cambio si la dejo aquí tarda 30 secs.

In [743]:
def rellenar_orga_sup(row):
    # Accedemos a los nulos que etán donde ambos campos son nulos.
    if pd.isna(row['id organizacion superior']) and pd.isna(row['organizacion superior']):
        id_org = row['id organizacion']
        if id_org in diccionario_orga_a_sup:
            row['id organizacion superior'], row['organizacion superior'] = diccionario_orga_a_sup[id_org]
    return row

df = df.apply(rellenar_orga_sup, axis=1)

In [744]:
df.isnull().sum()/df.shape[0]*100

Unnamed: 0                  0.000000
id organizacion superior    0.029234
organizacion superior       0.029234
id organizacion             2.447540
organizacion                3.399486
id unidad gestora           3.272319
unidad gestora              1.898337
categoria economica         1.849322
origen ingreso              3.743566
tipo ingreso                3.111144
valor previsto              5.000419
valor registrado            2.574414
valor recaudado             3.843448
porcentaje recaudado        2.351751
fecha recaudacion           2.314040
anio recaudacion            0.000000
dtype: float64

Hemos conseguido reducir los nulos a un 0.03% para id organizacion superioir y organizacion superior.

### 2- Gestión de los nulos de id organizacion y organizacion.

Vamos a seguir la misma estrategia que en el paso anterior.

In [745]:
len(df["id organizacion"].unique())

292

In [746]:
len(df["organizacion"].unique())

288

Hay más ids que organizaciones, lo cual impica que una misma organización puede tener 2 id, veámoslo.

In [747]:
lista_organizaciones = list(df["organizacion"].unique())
len(lista_organizaciones)

288

In [748]:
for orga in lista_organizaciones:
    filtro1 = df["id organizacion"].notnull()
    filtro2 = df["organizacion"] == orga
    if len(df[filtro1 & filtro2]["id organizacion"].unique()) > 1:
        print(f"\n{orga}")
        print(df[filtro1 & filtro2]["id organizacion"].unique())


Agência Nacional de Transportes Aquaviários
[68201. 39251.]

Fundo Nacional Antidrogas
[30912. 20117.]

Fundo Nacional do Idoso
[64902. 30914.]

Fundo Nacional para a Criança e o Adolescente
[64901. 30913.]


Efectivamente vemos que hay tres organizaciones con dos id. Vamos a ver que está pasando.

- Para la Agência Nacional de Transportes Aquaviários vemos que el id 39251 solo seusa en 5 filas y todas son de 2013 por lo que puede ser que sea un id viejo y haya transitado hacia el nuevo que es 68201, por lo que vamos a cambiar el id viejo por el nuevo para homogeneizar los datos.

In [749]:
filtro1_1 = df["id organizacion"] == 68201
df[filtro1_1].shape
df[filtro1_1].head(1)

,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
3304,3304,39000.0,Ministério da Infraestrutura,68201.0,Agência Nacional de Transportes Aquaviários,682010.0,AGENCIA NACIONAL DE TRANSPORTES AQUAVIARIOS,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",0.0,0.0,18958.86,0.0,2013-12-31,2013.0


In [750]:
filtro1_2 = df["id organizacion"] == 39251
df[filtro1_2].shape
df[filtro1_2].head(1)

,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
3307,3307,39000.0,Ministério da Infraestrutura,39251.0,Agência Nacional de Transportes Aquaviários,393002.0,AGENCIA NACIONAL DE TRANSPORTES AQUAVIARIOS,NaN,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",0.0,0.0,8481.37,0.0,2013-12-31,2013.0


In [751]:
df["id organizacion"] = df["id organizacion"].apply(sl.cambio_id_39251)

In [752]:
filtro1 = df["id organizacion"] == 39251
df[filtro1].shape

(0, 16)

- Para la Fundo Nacional Antidrogas pasa algo similar, sin embargo, el id 20117 aparece 20 veces y de forma aleatoria pues aparece en 2013, 2014, 2015, 2016 y hasta 2017, pero no a partir de 2017, lo que puede indicar que se fue transicionando de id durante varios años usandose dos a la vez hasta que en 2017 se dejó de usar. Además también se observa que para ambos ids a pesar de tener la misma unidad gestora también tienen ids de unidad gestora distintos.

IMPORTANTE:

He vuelto sobre mis pasos y me he dado cuenta de que la organización superior para los distintos id es distinta, por lo que si los unificara todos es como si estuviera perdiendo información ya que aunque la organización sea la misma si tiene el id de organización 30912 quiere decir que su organización superior es Ministério da Justiça e Segurança Pública y si tiene el id 20117 su organización superioir es Presidência da República. Me he dado cuenta de que pasa lo mismo también para los dos siguientes. En mi opinión unificar ambos id y dejar el que más aparece implicaría en cierta forma perder información, ya que si yo tengo un valor NaN para orga superioir e id de orga superior pero con id de organización 20117 se puede inferir que la organización superior es Presidência da República. Pero si unificasemos los id y pusiesemos para la organización Fundo Nacional Antidroga el id 30912 si tenemos un valor en NaN para id de orga superior y orga superior no podemos inferir si era Presidência da República o Ministério da Justiça e Segurança Pública. Pero como hemos visto que solo tenemos a NaN un 1% de las filas esto lo gestionaré si tuviera tiempo más adelante.

In [753]:
filtro2_1 = df["id organizacion"] == 30912
print(df[filtro2_1].shape)
print(df[filtro2_1]["id unidad gestora"].unique())
print(df[filtro2_1]["unidad gestora"].unique())
df[filtro2_1].head(1)

(7399, 16)
[200246.     nan]
['FUNDO NACIONAL ANTIDROGAS' nan]


,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
3447,3447,30000.0,Ministério da Justiça e Segurança Pública,30912.0,Fundo Nacional Antidrogas,200246.0,FUNDO NACIONAL ANTIDROGAS,Receitas Correntes,NaN,"Multas administrativas, contratuais e judicia",0.0,0.0,44.79,0.0,2013-12-31,2013.0


In [754]:
filtro2_2 = df["id organizacion"] == 20117
print(df[filtro2_2].shape)
print(df[filtro2_2]["id unidad gestora"].unique())
print(df[filtro2_2]["unidad gestora"].unique())
df[filtro2_2].head(1)

(20, 16)
[110246.]
['FUNDO NACIONAL ANTIDROGAS']


,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
4481,4481,20000.0,Presidência da República,20117.0,Fundo Nacional Antidrogas,110246.0,FUNDO NACIONAL ANTIDROGAS,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,0.0,0.0,4819.4,0.0,2013-12-31,2013.0


In [755]:
df["id organizacion"] = df["id organizacion"].apply(sl.cambio_id_20117)

In [756]:
filtro2 = df["id organizacion"] == 20117
df[filtro2].shape

(0, 16)

- Continuamos con los id de Fundo Nacional do Idoso. De nuevo ocurre lo mismo, el id 64902 aparece solo 99 veces y hasta 2017, el id 30914 aparece 223 y a partir de 2017 lo que indica un camio de id, siendo el actual 30914. De nuevo los id de unidad gestora son distintos aunque la unidad gestora sea la misma.


In [757]:
filtro3_1 = df["id organizacion"] == 64902
print(df[filtro3_1].shape)
df[filtro3_1].head(1)

(100, 16)


,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
3784,3784,57000.0,"Ministério das Mulheres, Igualdade Racial, da",64902.0,Fundo Nacional do Idoso,207001.0,FUNDO NACIONAL DO IDOSO,Receitas Correntes,Transferências Correntes,Transferências dos Municípios e de suas Entid,3000000.0,0.0,5850000.0,195.0,2013-12-31,2013.0


In [758]:
filtro3_2 = df["id organizacion"] == 30914
print(df[filtro3_2].shape)
df[filtro3_2].head(1)

(223, 16)


,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
359704,359741,81000.0,"Ministério da Mulher, Família e Direitos Huma",30914.0,Fundo Nacional do Idoso,307002.0,FUNDO NACIONAL DO IDOSO,Receitas Correntes,Receita Patrimonial,Valores Mobiliários,NaN,0.0,165329.77,0.0,2017-03-30,2017.0


In [759]:
df["id organizacion"] = df["id organizacion"].apply(sl.cambio_id_64902)

In [760]:
filtro3 = df["id organizacion"] == 64902
df[filtro2].shape

(0, 16)

- Terminamos con Fundo Nacional para a Criança e o Adolescente, en este caso ambos id se usn en 2021, el 64901 se empieza a usar en 2013 mientras que es 30913 se usa a apartir de 2017, por lo que vamos a dejar 30913.

In [761]:
filtro4_1 = df["id organizacion"] == 64901
print(df[filtro4_1].shape)
filtro4_2 = df["id organizacion"] == 30913
print(df[filtro4_2].shape)

(180, 16)
(722, 16)


In [762]:
df[filtro4_2].head(1)

,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
359752,359789,81000.0,"Ministério da Mulher, Família e Direitos Huma",30913.0,Fundo Nacional para a Criança e o Adolescente,307001.0,FUNDO NACIONAL PARA A CRIANCA E O ADOLESCENTE,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,0.0,0.0,15648.75,NaN,2017-06-20,2017.0


In [763]:
df[filtro4_1].head(1)

,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
3785,3785,57000.0,"Ministério das Mulheres, Igualdade Racial, da",64901.0,Fundo Nacional para a Criança e o Adolescente,110244.0,FUNDO NACIONAL PARA A CRIANCA E O ADOLESCENTE,Receitas Correntes,Transferências Correntes,Transferências dos Municípios e de suas Entid,24290692.0,NaN,9277740.79,38.0,2013-12-31,2013.0


In [764]:
df["id organizacion"] = df["id organizacion"].apply(sl.cambio_id_64901)

In [765]:
filtro4 = df["id organizacion"] == 64901
df[filtro4].shape

(0, 16)

In [766]:
#Comprobamos que ya solo hay un id por organización
flag = 0
for orga in lista_organizaciones:
    filtro1 = df["id organizacion"].notnull()
    filtro2 = df["organizacion"] == orga
    if len(df[filtro1 & filtro2]["id organizacion"].unique()) > 1:
        flag=1
        print(f"\n{orga}")
        print(df[filtro1 & filtro2]["id organizacion"].unique())
print(flag)

0


Ya podemos hacer un diccionario para rellenar valores.

In [767]:
lista_id_organizaciones = list(df["id organizacion"].unique())
print(len(lista_id_organizaciones))

288


In [768]:
dic_id_organizacion = {}
for id in lista_id_organizaciones:
    try:
        filtro1 = df["id organizacion"] == float(id)
        filtro2 = df["organizacion"].notnull()
        dic_id_organizacion[float(id)]=df[filtro1 & filtro2]["organizacion"].unique()[0]
    except:
        pass

In [769]:
df["id organizacion"].isnull().sum()

np.int64(25117)

In [770]:
# filas que van a quedar sin corregir.
filtro1 = df["id organizacion"].isnull()
filtro2 = df["organizacion"].isnull()
df[filtro1 & filtro2].shape

(905, 16)

In [771]:
df["organizacion"].fillna(df["id organizacion"].map(dic_id_organizacion), inplace=True)

C:\Users\Elena\AppData\Local\Temp\ipykernel_13844\3881576057.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["organizacion"].fillna(df["id organizacion"].map(dic_id_organizacion), inplace=True)


In [772]:
orga_null = df[df["organizacion"].isnull()].shape[0]
orga_sin_corregir = orga_null/df.shape[0]*100
print(f"Han quedado a null {orga_null}, es decir, un {np.round(orga_sin_corregir,2)}% del total de las organizaciones.")

Han quedado a null 905, es decir, un 0.09% del total de las organizaciones.


In [773]:
df[df["id organizacion"].isnull()].shape[0]

25117

Como podemos ver ids hay 25117 a null los cuales podemos corregir al rededor de unos 24200 a partir del nombre de la organizacion.

In [774]:
#Invertimos el diccionario
dic_orga_id = {valor: clave for clave, valor in dic_id_organizacion.items()}

In [775]:
df["id organizacion"].fillna(df["organizacion"].map(dic_orga_id), inplace=True)

C:\Users\Elena\AppData\Local\Temp\ipykernel_13844\2430629590.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["id organizacion"].fillna(df["organizacion"].map(dic_orga_id), inplace=True)


In [776]:
df[df["id organizacion"].isnull()].shape[0]

905

Comprobamos cuantos null hay ahora y efectivamente se han reducido hasta 905, los mismo que para los nombres de las organizaciones ya que son todas las filas que tienen ambos campos a null. Aunque al representar un 0.1% y al no haber forma de arreglarlo puede ser despreciable.

### 3- Gestión de los nulos de id unidad gestora y unidad gestora.

Vamos a seguir la misma estrategia que en el paso anterior.

In [777]:
len(df["id unidad gestora"].unique())

365

In [778]:
len(df["unidad gestora"].unique())

357

Como esperábamos, hay más ids que unidades gestoras.

In [779]:
lista_gestoras = list(df["unidad gestora"].unique())
len(lista_gestoras)

357

In [780]:
for gestora in lista_gestoras:
    filtro1 = df["id unidad gestora"].notnull()
    filtro2 = df["unidad gestora"] == gestora
    if len(df[filtro1 & filtro2]["id unidad gestora"].unique()) > 1:
        print(f"\n{gestora}")
        print(df[filtro1 & filtro2]["id unidad gestora"].unique())


SETORIAL PROG.ORCAMENTARIA E FINANCEIRA-SFB
[440088. 130214.]

COORDENACAO-GERAL DE ORCAMENTO E FINANCAS
[240102. 410002.]

DIRETORIA DE FINANCAS-SISTEMA PARA O PAIS/MM
[673001. 873001.]

AGENCIA NACIONAL DE TRANSPORTES AQUAVIARIOS
[682010. 393002.]

FUNDO NACIONAL ANTIDROGAS
[200246. 110246.]

FUNDO NACIONAL DO IDOSO
[207001. 307002.]

FUNDO NACIONAL PARA A CRIANCA E O ADOLESCENTE
[110244. 307001.]

COORDENACAO-GERAL DE RECURSOS LOGISTICOS
[410003. 240101.]


- Para SETORIAL PROG.ORCAMENTARIA E FINANCEIRA-SFB, el id 130214 solo aparece 7 veces en 2019, lo reemplazamos.

In [781]:
filtro1_1 = df["id unidad gestora"] == 440088
print(df[filtro1_1].shape)
df[filtro1_1].head(1)

filtro1_2 = df["id unidad gestora"] == 130214
print(df[filtro1_2].shape)
df[filtro1_2].head(1)


(584, 16)
(7, 16)


,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
573028,573088,22000.0,"Ministério da Agricultura, Pecuária e Abastec",44208.0,Serviço Florestal Brasileiro,130214.0,SETORIAL PROG.ORCAMENTARIA E FINANCEIRA-SFB,Receitas Correntes,Receita Patrimonial,Exploração de recursos naturais,0.0,0.0,11902.8,0.0,2019-04-29,2019.0


In [782]:
df["id unidad gestora"] = df["id unidad gestora"].apply(sl.cambio_id_130214)

In [783]:
filtro1 = df["id unidad gestora"] == 130214
df[filtro1].shape

(0, 16)

A partir de aqui he implementado la estrategia de que el id que aparezca en más filas es el que prevalece, me gustaría pararme a analizarlo en mayor profundidad pero no me da tiempo.

- COORDENACAO-GERAL DE ORCAMENTO E FINANCAS

In [784]:
filtro2_1 = df["id unidad gestora"] == 240102
print(df[filtro2_1].shape)
df[filtro2_1].head(1)

filtro2_2 = df["id unidad gestora"] == 410002
print(df[filtro2_2].shape)
df[filtro2_2].head(1)

df["id unidad gestora"] = df["id unidad gestora"].apply(sl.cambio_id_410002)

filtro2 = df["id unidad gestora"] == 410002
df[filtro2].shape


(5369, 16)
(290, 16)


(0, 16)

- DIRETORIA DE FINANCAS-SISTEMA PARA O PAIS/MM


In [785]:
filtro3_1 = df["id unidad gestora"] == 673001
print(df[filtro3_1].shape)
df[filtro3_1].head(1)

filtro3_2 = df["id unidad gestora"] == 873001
print(df[filtro2_2].shape)
df[filtro3_2].head(1)

df["id unidad gestora"] = df["id unidad gestora"].apply(sl.cambio_id_873001)

filtro3 = df["id unidad gestora"] == 873001
df[filtro3].shape

(18610, 16)
(290, 16)


(0, 16)

- AGENCIA NACIONAL DE TRANSPORTES AQUAVIARIOS


In [786]:
filtro4_1 = df["id unidad gestora"] == 682010
print(df[filtro4_1].shape)
df[filtro4_1].head(1)

filtro4_2 = df["id unidad gestora"] == 393002
print(df[filtro4_2].shape)
df[filtro4_2].head(1)

df["id unidad gestora"] = df["id unidad gestora"].apply(sl.cambio_id_393002)

filtro4 = df["id unidad gestora"] == 393002
df[filtro4].shape

(2096, 16)
(5, 16)


(0, 16)

- FUNDO NACIONAL ANTIDROGAS


In [787]:
filtro5_1 = df["id unidad gestora"] == 200246
print(df[filtro5_1].shape)
df[filtro5_1].head(1)

filtro5_2 = df["id unidad gestora"] == 110246
print(df[filtro5_2].shape)
df[filtro5_2].head(1)

df["id unidad gestora"] = df["id unidad gestora"].apply(sl.cambio_id_110246)

filtro5 = df["id unidad gestora"] == 110246
df[filtro5].shape

(7323, 16)
(20, 16)


(0, 16)

- FUNDO NACIONAL DO IDOSO


Me he dado cuenta de que esto lo podía haber automatizado antes y de una forma aún más eficiente, pero como ya me quedan tres no voy a cambiarlo todo.

In [788]:
num1=207001
num2=307002

filtro1 = df["id unidad gestora"] == num1
valor1 = df[filtro1].shape[0]
print(f"Para el {num1} tenemos un shape de {df[filtro1].shape}")

filtro2 = df["id unidad gestora"] == num2
valor2 = df[filtro2].shape[0]
print(f"Para el {num2} tenemos un shape de {df[filtro2].shape}")

if valor1 > valor2:
    print(f" Cambiamos {num2} --> {num1}")
    df["id unidad gestora"] = df["id unidad gestora"].apply(lambda id: num1 if id == num2 else id)
    filtro = df["id unidad gestora"] == num2
    print(df[filtro].shape)

else:
    print(f" Cambiamos {num1} --> {num2}")
    df["id unidad gestora"] = df["id unidad gestora"].apply(lambda id: num2 if id == num1 else id)
    filtro = df["id unidad gestora"] == num1
    print(df[filtro].shape)



Para el 207001 tenemos un shape de (85, 16)
Para el 307002 tenemos un shape de (226, 16)
 Cambiamos 207001 --> 307002
(0, 16)


- FUNDO NACIONAL PARA A CRIANCA E O ADOLESCENTE

In [789]:
num1=110244
num2=307001

filtro1 = df["id unidad gestora"] == num1
valor1 = df[filtro1].shape[0]
print(f"Para el {num1} tenemos un shape de {df[filtro1].shape}")

filtro2 = df["id unidad gestora"] == num2
valor2 = df[filtro2].shape[0]
print(f"Para el {num2} tenemos un shape de {df[filtro2].shape}")

if valor1 > valor2:
    print(f" Cambiamos {num2} --> {num1}")
    df["id unidad gestora"] = df["id unidad gestora"].apply(lambda id: num1 if id == num2 else id)
    filtro = df["id unidad gestora"] == num2
    print(df[filtro].shape)

else:
    print(f" Cambiamos {num1} --> {num2}")
    df["id unidad gestora"] = df["id unidad gestora"].apply(lambda id: num2 if id == num1 else id)
    filtro = df["id unidad gestora"] == num1
    print(df[filtro].shape)

Para el 110244 tenemos un shape de (161, 16)
Para el 307001 tenemos un shape de (728, 16)
 Cambiamos 110244 --> 307001
(0, 16)


- COORDENACAO-GERAL DE RECURSOS LOGISTICOS


In [790]:
num1=410003
num2=240101

filtro1 = df["id unidad gestora"] == num1
valor1 = df[filtro1].shape[0]
print(f"Para el {num1} tenemos un shape de {df[filtro1].shape}")

filtro2 = df["id unidad gestora"] == num2
valor2 = df[filtro2].shape[0]
print(f"Para el {num2} tenemos un shape de {df[filtro2].shape}")

if valor1 > valor2:
    print(f" Cambiamos {num2} --> {num1}")
    df["id unidad gestora"] = df["id unidad gestora"].apply(lambda id: num1 if id == num2 else id)
    filtro = df["id unidad gestora"] == num2
    print(df[filtro].shape)

else:
    print(f" Cambiamos {num1} --> {num2}")
    df["id unidad gestora"] = df["id unidad gestora"].apply(lambda id: num2 if id == num1 else id)
    filtro = df["id unidad gestora"] == num1
    print(df[filtro].shape)

Para el 410003 tenemos un shape de (3, 16)
Para el 240101 tenemos un shape de (3, 16)
 Cambiamos 410003 --> 240101
(0, 16)


Comprobamos que solo haya un id por unidad gestora:

In [791]:
flag=0
for gestora in lista_gestoras:
    filtro1 = df["id unidad gestora"].notnull()
    filtro2 = df["unidad gestora"] == gestora
    if len(df[filtro1 & filtro2]["id unidad gestora"].unique()) > 1:
        flag=1
        print(f"\n{gestora}")
        print(df[filtro1 & filtro2]["id unidad gestora"].unique())
print(flag)

0


Ya podemos hacer un diccionario para rellenar los valores:

In [792]:
lista_id_gestoras = list(df["id unidad gestora"].unique())
print(len(lista_id_gestoras))

357


In [793]:
dic_id_gestoras = {}
for id in lista_id_gestoras:
    try:
        filtro1 = df["id unidad gestora"] == float(id)
        filtro2 = df["unidad gestora"].notnull()
        dic_id_gestoras[float(id)]=df[filtro1 & filtro2]["unidad gestora"].unique()[0]
    except:
        pass

In [794]:
df["id unidad gestora"].isnull().sum()

np.int64(33581)

In [795]:
# filas que van a quedar sin corregir.
filtro1 = df["id unidad gestora"].isnull()
filtro2 = df["unidad gestora"].isnull()
df[filtro1 & filtro2].shape

(614, 16)

In [796]:
df["unidad gestora"].fillna(df["id unidad gestora"].map(dic_id_gestoras), inplace=True)

C:\Users\Elena\AppData\Local\Temp\ipykernel_13844\2383688579.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["unidad gestora"].fillna(df["id unidad gestora"].map(dic_id_gestoras), inplace=True)


In [797]:
gestora_null = df[df["unidad gestora"].isnull()].shape[0]
gestora_sin_corregir = gestora_null/df.shape[0]*100
print(f"Han quedado a null {gestora_null}, es decir, un {np.round(gestora_sin_corregir,2)}% del total de las filas en unidades gestoras.")

Han quedado a null 614, es decir, un 0.06% del total de las filas en unidades gestoras.


In [798]:
df[df["id unidad gestora"].isnull()].shape[0]

33581

Como podemos ver ids hay 33581 a null los cuales podemos corregir alrededor de unos 33000 a partir del nombre de la organizacion.

In [799]:
#Invertimos el diccionario
dic_gestora_id = {valor: clave for clave, valor in dic_id_gestoras.items()}

In [800]:
df["id unidad gestora"].fillna(df["unidad gestora"].map(dic_gestora_id), inplace=True)

C:\Users\Elena\AppData\Local\Temp\ipykernel_13844\105797111.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["id unidad gestora"].fillna(df["unidad gestora"].map(dic_gestora_id), inplace=True)


In [801]:
df[df["id unidad gestora"].isnull()].shape[0]

614

Comprobamos cuantos null hay ahora y efectivamente se han reducido hasta 614, los mismo que para los nombres de las unidades gestoras ya que son todas las filas que tienen ambos campos a null. Aunque al representar un 0.06% y al no haber forma de arreglarlo puede ser despreciable.

### 4- Gestión de los nulos de categoría económica.

In [802]:
df.isnull().sum()/df.shape[0]*100

Unnamed: 0                  0.000000
id organizacion superior    0.029234
organizacion superior       0.029234
id organizacion             0.088188
organizacion                0.088188
id unidad gestora           0.059832
unidad gestora              0.059832
categoria economica         1.849322
origen ingreso              3.743566
tipo ingreso                3.111144
valor previsto              5.000419
valor registrado            2.574414
valor recaudado             3.843448
porcentaje recaudado        2.351751
fecha recaudacion           2.314040
anio recaudacion            0.000000
dtype: float64

In [803]:
df["categoria economica"].value_counts()

categoria economica
Receitas Correntes                           961445
Receitas de Capital                           29523
Receitas Correntes - intra-orçamentárias      15926
Sem informação                                  252
Receitas de Capital - intra-orçamentárias        90
Name: count, dtype: int64

In [804]:
df["categoria economica"].isna().sum()

np.int64(18978)

In [805]:
df["categoria economica"].isna().sum()/df.shape[0]*100

np.float64(1.849321876333786)

Tenemos casi un 2% de nulos en esta columna, como no tenemos información con la que rellenarlos y sabemos que dentro de categorías exite una que es "Sem informação", en vez de dejar los nulos vamos a meterlos en esta categoría.

In [806]:
df["categoria economica"].fillna("Sem informação", inplace=True)

C:\Users\Elena\AppData\Local\Temp\ipykernel_13844\2181394413.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["categoria economica"].fillna("Sem informação", inplace=True)


In [807]:
df["categoria economica"].value_counts()

categoria economica
Receitas Correntes                           961445
Receitas de Capital                           29523
Sem informação                                19230
Receitas Correntes - intra-orçamentárias      15926
Receitas de Capital - intra-orçamentárias        90
Name: count, dtype: int64

In [808]:
df["categoria economica"].isna().sum()

np.int64(0)

### 5- Gestión de los nulos de origen ingreso.

In [809]:
df["origen ingreso"].unique()

array(['Outras Receitas Correntes', 'Receita de Serviços',
       'Transferências de Capital', 'Alienação de Bens',
       'Receita Agropecuária', 'Transferências Correntes',
       'Receita Patrimonial', 'Operações de Crédito',
       'Impostos, Taxas e Contribuições de Melhoria', nan,
       'Amortizações de Empréstimos', 'Contribuições',
       'Receita Industrial', 'Outras Receitas de Capital',
       'Receitas Correntes - a classificar', 'Sem informação'],
      dtype=object)

In [810]:
df["origen ingreso"].isna().sum()/df.shape[0]*100

np.float64(3.7435661567665224)

Tenemos un 3.8% de origenes de ingreso a nulo, como no tenemos información sidicinet como para saber de forma segura de donde vienen vamos a ponerlos como "Sem informação"

In [811]:
df["origen ingreso"].fillna("Sem informação", inplace=True)

C:\Users\Elena\AppData\Local\Temp\ipykernel_13844\886053175.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["origen ingreso"].fillna("Sem informação", inplace=True)


In [812]:
df["origen ingreso"].isna().sum()

np.int64(0)

### 6- Gestión de los nulos de tipo ingreso.

In [813]:
df["tipo ingreso"].isna().sum()/df.shape[0]*100

np.float64(3.1111444591478974)

De nuevo, en la columna "tipo ingreso" tenemos un 3.13% de los datos nulos, como hay muchas categorías y no sabems a cual pertenecen los nulos antes que darlos a NaN les metemos en "Sem informação".

In [814]:
df["tipo ingreso"].unique()

array(['Bens, Direitos e Valores Incorporados ao Patr',
       'Indenizações, restituições e ressarcimentos',
       'Multas administrativas, contratuais e judicia',
       'Demais receitas correntes', 'Receita de Serviços',
       'Transferências de Instituições Privadas',
       'Alienação de bens móveis',
       'Receita da produção animal e derivados',
       'Transferências dos Municípios e de suas Entid',
       'Alienação de bens imóveis', 'Transferências de Pessoas Físicas',
       'Valores Mobiliários',
       'Exploração do patrimônio imobiliário do Estad',
       'Operações de crédito - mercado interno', 'Taxas', nan,
       'Delegação de Serviços Públicos Mediante Conce',
       'Outras receitas agropecuárias', 'Amortizações de Empréstimos',
       'Contribuições econômicas', 'Transferências de Convênios',
       'Indenizações e restituições',
       'Receitas da indústria de transformação', 'Contribuições sociais',
       'Receita dívida ativa alienação de estoques de',
  

In [815]:
df["tipo ingreso"].fillna("Sem informação", inplace=True)

C:\Users\Elena\AppData\Local\Temp\ipykernel_13844\1751979089.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["tipo ingreso"].fillna("Sem informação", inplace=True)


In [816]:
df["tipo ingreso"].isna().sum()

np.int64(0)

### 6- Gestión de los nulos de valores previsto, registrado, recaudado y porcentaje.


In [817]:
df.columns

Index(['Unnamed: 0', 'id organizacion superior', 'organizacion superior',
       'id organizacion', 'organizacion', 'id unidad gestora',
       'unidad gestora', 'categoria economica', 'origen ingreso',
       'tipo ingreso', 'valor previsto', 'valor registrado', 'valor recaudado',
       'porcentaje recaudado', 'fecha recaudacion', 'anio recaudacion'],
      dtype='object')

In [818]:
df.isnull().sum()[["valor previsto", "valor registrado", "valor recaudado", "porcentaje recaudado"]]

valor previsto          51315
valor registrado        26419
valor recaudado         39442
porcentaje recaudado    24134
dtype: int64

In [819]:
(df.isnull().sum()/df.shape[0]*100)[["valor previsto", "valor registrado", "valor recaudado", "porcentaje recaudado"]]

valor previsto          5.000419
valor registrado        2.574414
valor recaudado         3.843448
porcentaje recaudado    2.351751
dtype: float64

In [820]:
filtro1 = df["valor previsto"].isnull()
filtro2 = df["valor registrado"].isnull()
filtro3 = df["valor recaudado"].isnull()
filtro4 = df["porcentaje recaudado"].isnull()

filtro5 = df["valor previsto"]==0
filtro6 = df["valor registrado"]==0
filtro7 = df["valor recaudado"]==0
filtro8 = df["porcentaje recaudado"]==0

filtros_comb = filtro1 & filtro2 & filtro3 & filtro4

print(df[filtro1].shape)
print(df[filtro2].shape)
print(df[filtro3].shape)
print(df[filtro4].shape)
print(f" Los 3 campos a null: {df[filtro1 & filtro2 & filtro3].shape}")
print(f" Los 4 campos a null: {df[filtros_comb].shape}")

(51315, 16)
(26419, 16)
(39442, 16)
(24134, 16)
 Los 3 campos a null: (54, 16)
 Los 4 campos a null: (0, 16)


- Para las filas que no sea null su valor recaudado y valor previsto podemos calcular su porcentaje.

In [821]:
df[filtro5 & ~filtro3 & filtro4 & df["valor previsto"]!=0].head(2)

,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion


In [822]:
def calcular_porcentaje(fila):
    if pd.isnull(fila['porcentaje recaudado']) and pd.notnull(fila['valor previsto']) and pd.notnull(fila['valor recaudado']) and fila['valor previsto']!=0:
        return (fila['valor recaudado'] / fila['valor previsto']) * 100
    return fila['porcentaje recaudado']

df['porcentaje recaudado'] = df.apply(calcular_porcentaje, axis=1)

In [823]:
# Algo a descendido, aunque no mucho.
(df.isnull().sum()/df.shape[0]*100)["porcentaje recaudado"]

np.float64(2.3008846108121697)

- Las filas que tengan los valores a null y el porcentaje a 0 no aportan nada de valor y podemos quitarlas. Lo mismo ocurre si de las 4 columnas hay 3 a null y la restante a 0, esas las quitamos. O las columnas que tienen 3 ceros y un nan, los 0 si que nos aportaría un cierto valor pero no es nada reaslita que 3 de las 4 columnas sean 0, ahí claramente está habiendo un problema a la hora de recoger los datos que sería de vital importancial solucionarlo.

In [824]:
df.shape[0]

1026214

In [825]:
# filtro: nan nan nan 0
f1 = filtro1 & filtro2 & filtro3 & filtro8
# filtro: nan nan 0 nan
f2 = filtro1 & filtro2 & filtro7 & filtro4
# filtro: nan 0 nan nan
f3 = filtro1 & filtro6 & filtro3 & filtro4
# filtro: 0 nan nan nan
f4 = filtro5 & filtro2 & filtro3 & filtro4

# filtro: nan 0 0 0
f5 = filtro1 & filtro6 & filtro7 & filtro8
# filtro: 0 nan 0 0
f6 = filtro5 & filtro2 & filtro7 & filtro8
# filtro: 0 0 nan 0
f7 = filtro5 & filtro6 & filtro3 & filtro8
# filtro: 0 0 0 nan
f8 = filtro5 & filtro6 & filtro7 & filtro4
# filtro: 0 0 0 0 (podría dejarlos pero es que estos valores no sería realistas y harían ruido)
f9 = filtro5 & filtro6 & filtro7 & filtro8

filtros_combo = f1 | f2 | f3 | f4 | f5 | f6 | f7 | f8 | f9


In [826]:
df[f1|f2|f3|f4|f5|f6|f7|f8|f9].shape

(39012, 16)

In [827]:
df_final = df[~filtros_combo] #invierto el filtro para quedarme con las filas contrarias a las que lo cumplen.

In [828]:
df_final.shape

(987202, 16)

In [829]:
(f"Nos hemos quedado con un {np.round(df_final.shape[0]/df.shape[0]*100, 2)}% de la tabla original")

'Nos hemos quedado con un 96.2% de la tabla original'

In [830]:
df_final.isnull().sum()/df_final.shape[0]*100

Unnamed: 0                  0.000000
id organizacion superior    0.029680
organizacion superior       0.029680
id organizacion             0.087723
organizacion                0.087723
id unidad gestora           0.059967
unidad gestora              0.059967
categoria economica         0.000000
origen ingreso              0.000000
tipo ingreso                0.000000
valor previsto              5.127826
valor registrado            2.644241
valor recaudado             0.475181
porcentaje recaudado        2.376515
fecha recaudacion           2.314217
anio recaudacion            0.000000
dtype: float64

In [831]:
df_final.dtypes

Unnamed: 0                           int64
id organizacion superior           float64
organizacion superior               object
id organizacion                    float64
organizacion                        object
id unidad gestora                  float64
unidad gestora                      object
categoria economica                 object
origen ingreso                      object
tipo ingreso                        object
valor previsto                     float64
valor registrado                   float64
valor recaudado                    float64
porcentaje recaudado               float64
fecha recaudacion           datetime64[ns]
anio recaudacion                   float64
dtype: object

### 7- Gestión de los nulos de fecha recaudación.


Un 2.3% de los datos de la columna son nulos, no hay forma de reemplazarlos por una fecha, pero por lo menos tenemos la columna de año y podriamos borara esta pero si quisieramos acceder a los meses ya no se podría por lo que por ahora vamos a dejarla como esta.

## Operaciones finales

In [832]:
df_final.reset_index(inplace=True)

In [833]:
df_final = df_final.drop(columns=['Unnamed: 0', 'index'])


In [834]:
df_final.head()

,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
0,63000.0,Advocacia-Geral da União,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",0.0,0.0,1297.13,0.0,2013-12-31,2013.0
1,63000.0,Advocacia-Geral da União,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",0.0,0.0,26666621.42,0.0,2013-12-31,2013.0
2,63000.0,Advocacia-Geral da União,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",0.0,0.0,301251.13,0.0,2013-12-31,2013.0
3,63000.0,Advocacia-Geral da União,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",0.0,0.0,1855.58,0.0,2013-12-31,2013.0
4,63000.0,Advocacia-Geral da União,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",0.0,0.0,52140.68,0.0,2013-12-31,2013.0


In [835]:
(df_final.isnull().sum()/df_final.shape[0]*100)

id organizacion superior    0.029680
organizacion superior       0.029680
id organizacion             0.087723
organizacion                0.087723
id unidad gestora           0.059967
unidad gestora              0.059967
categoria economica         0.000000
origen ingreso              0.000000
tipo ingreso                0.000000
valor previsto              5.127826
valor registrado            2.644241
valor recaudado             0.475181
porcentaje recaudado        2.376515
fecha recaudacion           2.314217
anio recaudacion            0.000000
dtype: float64

In [836]:
df_final.to_csv("../datos/tabla_limpia.csv")

# Conclusiones

1 - Hemos corregido los nombres de las organizaciones superiores a partir de los id y han quedado solo 10543 sin corregir lo que representa un 1.04% del total. Lo mismo hemos hecho para corregir los id a partir del nombre de la organización superioir, de nuevo queda un 1.04% sin corregir el cual representa las filas donde ambas columnas tienen valores nulos. porterioirmente hemos conseguido reducirlo a un 0.03%.

2 - Hemos corregido los nombres de las organizaciones a partir de los id y han quedado solo 905 sin corregir lo que representa un 0.1% del total. Lo mismo hemos hecho para corregir los id a partir del nombre de la organización, de nuevo queda un 0.1% sin corregir el cual representa las filas donde ambas columnas tienen valores nulos.

3 - Hemos corregido los nombres de las unidades gestoras a partir de los id y han quedado solo 614 sin corregir lo que representa un 0.06% del total. Lo mismo hemos hecho para corregir los id a partir del nombre de la unidad gestora, de nuevo queda un 0.06% sin corregir el cual representa las filas donde ambas columnas tienen valores nulos.

4 - Para las columnas 'categoria economica', 'origen ingreso', 'tipo ingreso', los nulos al no poder ser gestionados pues no sabemos que valor pueden tener los hemos asignado como "Sem informação" antes que dejarlos como nulos.

5 - Para las columnas de "valor previsto", "valor registrado", "valor recaudado", "porcentaje recaudado", aparecen una grandísima cantidad de valors nulos lo que nos imposibilita realizar un buen EDA. A modo de limpieza hemos eliminado una gran cantidad de datos que no aportaban valor, quedandonos con tan solo un 0.65% de la tabla original. Si esto fuera un caso real no tendría sentido continuar con el estudio ya que de los datos aportados solo nos serían útiles menos de un 1% (y ni eso). Por lo que le pediría al gobierno de Brasil que por favor revisara estos datos y me los devolviera con unos valores más fieles a la realidad. Pero por ahora continuaremos con lo que hemos calculado.

6 - Los nulos de la fecha de recaudación no puedo gestionarlos de ninguna manera por lo que los mantengo.
